In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline


print('import success')

import success


In [2]:
class Value:
    
    def __init__(self,data, _children=(), _op='', label=''):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0
        self._backward = lambda: None
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "supports only integer/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        
        return out
            
              
    
    def __rmul__(self, other):
        return self*other
    
    def __truediv__(self, other): # self / other
        return self * other**-1
    
    def __neg__(self):   # negation i.e -self
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)
    
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
            
        out._backward = _backward
            
        
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    

    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
            
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


    
    
print('success')

success


In [3]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f}"% (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name = uid + n._op, label = n._op)
            dot.edge(uid + n._op, uid)
            
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot


print('success')

success


In [ ]:
###tanh as whole 
## inputs x1,x1
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

## weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

## bias b
b = Value(6.8813735870195432, label='b')

## x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.backward()

In [ ]:
#draw_dot(o)

In [ ]:
###tanh broken
## inputs x1,x1
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

## weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

## bias b
b = Value(6.8813735870195432, label='b')

## x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
# -----
e = (2*n).exp()
o = (e-1)/(e+1) 
# -----
o.label = 'o'
o.backward()
draw_dot(o)

In [ ]:
import torch

x1 = torch.Tensor([2.0]).double()                     ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()                     ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()                    ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                     ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()       ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('___')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())


In [ ]:
o.data.item()

In [4]:
##need to import random first

class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
        
    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
    
class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP:
    
    def __init__(self, nin, nouts):
        sz =[nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
        
    

print('success')

success


In [6]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)
        

Value(data=0.8076708781515928)

In [6]:
n.parameters()

[Value(data=0.22785139825439038),
 Value(data=-0.5141526564271883),
 Value(data=-0.4830769013355878),
 Value(data=-0.4074592710102689),
 Value(data=0.8385879616955285),
 Value(data=-0.40347322003331976),
 Value(data=0.9537683027995205),
 Value(data=0.5278162271747115),
 Value(data=-0.07692842558711543),
 Value(data=0.7239665045035384),
 Value(data=-0.5940669132170848),
 Value(data=0.37315243650139096),
 Value(data=0.06355964921567425),
 Value(data=-0.8958566437465947),
 Value(data=-0.33179908136011527),
 Value(data=0.8738366072980717),
 Value(data=-0.48584988666252715),
 Value(data=0.815700044255427),
 Value(data=-0.1449936361991273),
 Value(data=-0.436241982818101),
 Value(data=0.39010844934569255),
 Value(data=-0.5997206180964183),
 Value(data=-0.7327926699198315),
 Value(data=0.781645735867621),
 Value(data=0.49940069095989936),
 Value(data=0.8051376901375569),
 Value(data=0.17949088035593275),
 Value(data=0.08338451269464842),
 Value(data=-0.8684776446094757),
 Value(data=0.3848083

In [19]:
len(n.parameters())

41

In [ ]:
draw_dot(n(x))

In [5]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [10]:
## does not work without start argument to initilize summing with Value object set to zero

for k in range(20):
    
    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum(((yout - ygt)**2 for ygt, yout in zip(ys, ypred)), start=Value(0))
    
    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    #update
    for p in n.parameters():
        p.data += -0.05 * p.grad
        
    print(k, loss.data)



0 0.042682512369833336
1 0.039774498135288106
2 0.037208627633266225
3 0.03492992919683871
4 0.032894365557668274
5 0.03106628280931118
6 0.029416534064838323
7 0.02792108018853014
8 0.026559933011818485
9 0.025316347860440683
10 0.02417619991257993
11 0.02312749772461157
12 0.022160000238832824
13 0.021264912662884283
14 0.020434643039182236
15 0.01966260593146944
16 0.018943062997002408
17 0.018270992661034786
18 0.01764198292169685
19 0.01705214266617995


In [11]:
ypred

[Value(data=0.9461627026081698),
 Value(data=-0.9599160983705597),
 Value(data=-0.9235141104942557),
 Value(data=0.9181655475218413)]

In [8]:
##Forward pass
## does not work without start argument to initilize summing with Value object set to zero
#ypred = [n(x) for x in xs]
#loss = sum(((yout - ygt)**2 for ygt, yout in zip(ys, ypred)), start=Value(0))

#print('loss',loss)

loss Value(data=5.044763282454489)
